In [0]:
import tensorflow as tf
import numpy as np
from google.colab import drive
import os

drive.mount('gdrive')
os.chdir('gdrive/My Drive/movie plot')

In [0]:
class TextCNN():
  def __init__(
    self,sequence_length,num_classes,vocab_size,embedding_size,filter_sizes,num_filters,embedding_matrix):

    self.input_x=tf.placeholder(tf.int32,[None,sequence_length],name="input_x")
    self.input_y=tf.placeholder(tf.float32,[None,num_classes],name="input_y")
    self.dropout_keep_prob=tf.placeholder(tf.float32,name="dropout_keep_prob")
    

    with tf.name_scope('embedding'):
        self.W=tf.Variable(embedding_matrix,name="W",dtype=tf.float32)
        self.embedded=tf.nn.embedding_lookup(self.W,self.input_x)
        self.embedded_expanded=tf.expand_dims(self.embedded,-1)
    
    pooled_outputs=[]
    for i,filter_size in enumerate(filter_sizes):
        with tf.name_scope("conv_maxpool-%s"%filter_size):
            filter_shape=[filter_size,embedding_size,1,num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")

            conv=tf.nn.conv2d(
                self.embedded_expanded,W,strides=[1,1,1,1],padding="VALID",name="conv"
            )
            h=tf.nn.relu(tf.nn.bias_add(conv,b),name="relu")
            pooled=tf.nn.max_pool(
                h,ksize=[1,sequence_length-filter_size+1,1,1],
                strides=[1,1,1,1],
                padding='VALID',
                name="pool"
            )
            pooled_outputs.append(pooled)

    num_filters_total=num_filters*len(filter_sizes)
    self.h_pool=tf.concat(pooled_outputs,3)
    self.h_pool_flat=tf.reshape(self.h_pool,[-1,num_filters_total])

    with tf.name_scope("dropout"):
      self.h_drop=tf.nn.dropout(self.h_pool_flat,self.dropout_keep_prob)

    with tf.name_scope("output"):
      W=tf.get_variable(
          "W",shape=[num_filters_total,num_classes],
          initializer=tf.contrib.layers.xavier_initializer()
      )
      b=tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
      self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
      self.predictions = tf.argmax(self.scores, 1, name="predictions")

    with tf.name_scope("loss"):
      losses=tf.nn.softmax_cross_entropy_with_logits(logits=self.scores,labels=self.input_y)
      self.loss=tf.reduce_mean(losses)

    with tf.name_scope("accuracy"):
      correct_predictions=tf.equal(self.predictions, tf.argmax(self.input_y, 1))
      self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [0]:
def train_step(sess,cnn,x_batch,y_batch,dropout_keep_prob,train_op,global_step):
  feed_dict={
      cnn.input_x:x_batch,
      cnn.input_y:y_batch,
      cnn.dropout_keep_prob:0.5
  }
  _, step,  loss, accuracy = sess.run(
        [train_op, global_step, cnn.loss, cnn.accuracy],
        feed_dict)
  print("step {}, loss {}, acc {}".format(step, loss, accuracy))

  return accuracy

In [0]:
def dev_step(sess,cnn,x_batch, y_batch, global_step):
    feed_dict = {
      cnn.input_x: x_batch,
      cnn.input_y: y_batch,
      cnn.dropout_keep_prob: 1.0
      }
    step,loss, accuracy = sess.run(
                      [global_step, cnn.loss, cnn.accuracy],
                      feed_dict)
    print("step {}, loss {:}, acc {:}".format(step, loss, accuracy))
    return accuracy

In [0]:
def load_data():
  import pandas as pd
  from sklearn.preprocessing import OneHotEncoder
  enc=OneHotEncoder()
  X_train=np.load("dataset/X_train.npy")
  y_train=np.load("dataset/y_train.npy")
  y_train=enc.fit_transform(y_train.reshape(-1,1)).toarray()
  X_val=np.load("dataset/X_val.npy")
  y_val=np.load("dataset/y_val.npy")
  y_val=enc.fit_transform(y_val.reshape(-1,1)).toarray()
  X_test=np.load("dataset/X_test.npy")
  y_test=np.load("dataset/y_test.npy")
  y_test=enc.fit_transform(y_test.reshape(-1,1)).toarray()

  return X_train.astype(int),y_train,X_val.astype(int),y_val,X_test.astype(int),y_test

In [0]:
def batch_iter(data, batch_size, num_epochs):
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_data = data[shuffle_indices]
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [0]:
def train(graph, x_train,y_train,x_val,y_val,X_test,y_test,embedding_matrix,l2_clip_threshold,filter_sizes,num_filters,num_epochs):
  with graph.as_default():
    sess=tf.Session()
    with sess.as_default():
      cnn=TextCNN(
          sequence_length=x_train.shape[1],
          num_classes=y_train.shape[1],
          vocab_size=20001,
          embedding_size=300,
          filter_sizes=filter_sizes,
          num_filters=num_filters,
          embedding_matrix=embedding_matrix
      )
    
    global_step=tf.Variable(0,name="global_step")

    optimizer=tf.train.AdamOptimizer(0.001)
    grads_and_vars=optimizer.compute_gradients(cnn.loss)
    clipped_grads_and_vars = [(tf.clip_by_norm(gv[0],clip_norm=l2_clip_threshold), gv[1]) if gv[0] is not None else(gv[0],gv[1]) for gv in grads_and_vars]
    train_op=optimizer.apply_gradients(grads_and_vars, global_step=global_step)



    loss_summary = tf.summary.scalar("loss", cnn.loss)
    acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)
    print("loss: "+str(loss_summary)+" acc: "+str(acc_summary))


    checkpoint_dir = "./cnn_checkpoint/cnn_dynamic_embedding_weights.ckpt"

    saver = tf.train.Saver(tf.global_variables(),checkpoint_dir, max_to_keep=max_num_checkpoints)

    sess.run(tf.global_variables_initializer())

    batches = batch_iter(
        list(zip(x_train, y_train)), batch_size, num_epochs)

    train_acc=0
    dev_acc=0

    for batch in batches:
        x_batch, y_batch = zip(*batch)
        train_acc = train_step(sess,cnn,x_batch, y_batch,dropout_keep_prob,train_op,global_step)
        current_step = tf.train.global_step(sess, global_step)
        if current_step % evaluate_every == 0:
            print("\nEvaluation:")
            dev_acc=dev_step(sess,cnn,x_val, y_val,global_step)
        if current_step % checkpoint_every == 0:
            path = saver.save(sess, checkpoint_dir)
            print("Saved model checkpoint to {}\n".format(path))
    
    print("***testing*****\n")

    test_acc = dev_step(sess,cnn,X_test,y_test,global_step)
    return test_acc,train_acc,dev_acc

In [0]:
X_train,y_train,X_val,y_val,X_test,y_test=load_data()

In [0]:
embedding_dim=300
#filter_sizes=[3,4,5]
#num_filters=100
dropout_keep_prob=0.5


batch_size=50
#num_epochs=10
evaluate_every=100
checkpoint_every=100
max_num_checkpoints=1
l2_clip_threshold=3

In [0]:

embedding_matrix=np.load("dataset/embedding_matrix.npy")
import csv
import time
best_test=0
for filter_sizes in [[3,4,5]]:
  for num_filters in [100]:
        graph=tf.Graph()
        test_acc,train_acc,dev_acc=train(graph,X_train,y_train,X_val,y_val,X_test,y_test,embedding_matrix,l2_clip_threshold,filter_sizes,num_filters,num_epochs)
        if test_acc>best_test:
          best_test=test_acc
          with open('dataset/dynamic_best.csv','a') as f:
            writer=csv.writer(f)
            t = time.localtime()
            current_time = time.strftime("%H:%M:%S", t)
            writer.writerow([test_acc,train_acc,dev_acc,filter_sizes,num_filters,num_epochs,str(current_time)])

with open('dataset/dynamic_best.csv','a') as f:
  writer=csv.writer(f)
  
  t = time.localtime()
  current_time = time.strftime("%H:%M:%S", t)
  writer.writerow([str(current_time)])